In [1]:
import torch

In [6]:
points = torch.tensor([[4, 1], [5,3], [2,1.0]])
points, points.storage()

(tensor([[4., 1.],
         [5., 3.],
         [2., 1.]]),
  4.0
  1.0
  5.0
  3.0
  2.0
  1.0
 [torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6])

points will report that its size is (3,2), but the storage is just a contiguous array.

We can index into a storage:

In [7]:
points.storage()[0]

4.0

and modifying the storage unsurprisingly modifies the tensor's view of it

In [9]:
points.storage()[0] = -99.0
points

tensor([[-99.,   1.],
        [  5.,   3.],
        [  2.,   1.]])

as we've seen before, methods which end in underscore modify the underlying storage; all others return a copy

In [10]:
points.zero_()
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## Tensor Metadata

tensors are defined by three pieces of data:
- **size** is a tuple indicating how many elements across each dimension the tensor represents
- **offset** is the index in the storage corresponding to the first element in the tensor
- **stride** is the number of elements in the storage that need to be skipped over to obtain the next element along each dimension

In [11]:
points[1].storage_offset(), points[1].size()

(2, torch.Size([2]))